# Kürzeste Wege auf Graphen
## Einleitung
In diesem Notebook werden wir uns mit dem Thema kürzeste Wege auf Graphen beschäftigen. Wir werden verschiedene Algorithmen kennenlernen, die uns helfen, den kürzesten Weg zwischen zwei Knoten in einem Graphen zu finden. Wir betrachten hierbei zunächst nur *ungerichtete* Graphen mit Gewichten.
## Graphen
Ein Graph ist eine Menge von Knoten (auch als Ecken oder Punkte bezeichnet) und eine Menge von Kanten (auch als Linien oder Verbindungen bezeichnet), die die Knoten miteinander verbinden. Ein Graph kann formal als Paar \( G = (V, E) \) dargestellt werden, wobei \( V \) die Menge der Knoten und \( E \) die Menge der Kanten ist. Jede Kante verbindet zwei Knoten und kann ein Gewicht haben, das die Kosten oder die Entfernung zwischen den beiden Knoten darstellt.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from utils import Pfad, Knoten, Knotenliste, dijkstra_graph
import random

keys = "SABCDEFGHIJKL"
m = {key:i for i, key in enumerate(keys)}

verbindungen = np.array([
    [1,4], # S
    [0,3], # A
    [2.5,3], # B
    [6,4], # C
    [0,2], # D
    [6,0], # E
    [0.5,0], # F
    [4,1], # G
    [2,1.5], # H
    [6,2.5], # I
    [8,2.5], # J
    [7, 2], # K
    [7,3], # L
])

adj_mat = np.zeros([len(verbindungen), len(verbindungen)])
def add_adj(val):
    adj_mat[m[val[0]], m[val[1]]] = int(val[2])
    adj_mat[m[val[1]], m[val[0]]] = int(val[2])
add_adj("SA7");add_adj("SB2");add_adj("SC3");add_adj("AD4");add_adj("BA3");add_adj("BD4");add_adj("BH5");add_adj("CL2");add_adj("DF5");add_adj("EG2");add_adj("EK5");add_adj("FH3");add_adj("GH2");add_adj("IL4");add_adj("IK4");add_adj("IJ6");add_adj("JK4");add_adj("JL4")
print(adj_mat)

def plot(data, adj):
    plt.figure()
    plt.scatter(data[:,0], data[:,1], zorder=3)
    for i in range(len(data)):
        plt.text(*(data[i]), keys[i], zorder=3)
        for j in range(i+1, len(data)):
            if adj[i,j]:
                plt.plot([data[i,0], data[j,0]], [data[i,1], data[j,1]], "k")
                plt.text(*((data[i]+data[j])/2), str(int(adj[i,j])), zorder=3)

    plt.axis("scaled")
    plt.show()

plot(verbindungen, adj_mat)

## Wegsuche in einem Graphen
Ein Weg in einem Graphen ist eine Sequenz von Knoten, die durch Kanten verbunden sind. Der kürzeste Weg zwischen zwei Knoten ist der Weg mit der geringsten Summe der Gewichte der Kanten, die den Weg bilden. Es gibt verschiedene Algorithmen, um den kürzesten Weg in einem Graphen zu finden:
- Zufällig durchsuchen
- "Nach Gefühl"-Suche
- Greedy Suche
- Dijkstra-Algorithmus
- A*-Algorithmus

### Zufällige Suche
Bei der zufälligen Suche durchlaufen wir zufällig den kompletten Graphen, bis wir beim Zielknoten angelangt sind. Offensichtlich ist das nicht besonders effizient, da wir im schlimmsten Fall jeden Knoten häufiger besuchen. Anders herum kann die zufällige Suche jedoch auch den kürzesten Weg finden, wenn wir Glück haben!

#### Pruning
Eine Möglichkeit, den zufälligen Pfad zu kürzen ist es, einfach die "Schlaufen" zu beseitigen. Das bedeutet, dass wir im fertigen Pfad nachsehen, ob ein bestimmter Knoten mehrmals vorkommt. Sollte dies der Fall sein, entfernen wir einfach alle Knoten in unserer Liste zwischen den beiden gleichen Knoten. Der dadurch entstehende Pfad ist kürzer und wahrscheinlicher auch der optimale Pfad.

In [ ]:
knotenliste = Knotenliste()
for i in range(len(verbindungen)):
    knotenliste.append(Knoten(keys[i], verbindungen[i]))

adj = ["SA7", "SB2", "SC3", "AD4", "BA3", "BD4", "BH5", "CL2", "DF5", "EG2", "EK5", "FH3", "GH2", "IL4", "IK4", "IJ6", "JK4", "JL4"]

for a in adj:
    knotenliste.verbinde(a)

knotenliste.draw()

random.seed(2)
pfad = Pfad()
pfad.randomwalk(knotenliste)
print("Ohne Pruning:")
print(pfad)

#### Bonusaufgabe:
Implementiere die Funktion `prune`, die eine Liste von Knoten als Eingabe erhält und alle Knoten entfernt, die mehr als einmal vorkommen. Die Funktion sollte den gekürzten Pfad zurückgeben.

In [ ]:
def prune(liste):
    out = []
    return out

liste = pfad.pfad
kostenliste = pfad.kostenliste
liste = prune(liste)
print("Mit Pruning:")
print(liste)

### "Nach Gefühl"-Suche
Bei der "Nach Gefühl"-Suche versuchen wir, den kürzesten Weg zu finden, indem wir immer den Knoten auswählen, der uns am nächsten zum Zielknoten erscheint. Hierbei gibt es keine bestimmten Regeln, wir arbeiten einfach "nach Gefühl".


In [ ]:
pfad = Pfad()
pfad.menschenwalk(knotenliste)
print(pfad)

### Greedy-Suche
Die "Greedy"-Suche ist eine Erweiterung der "Nach Gefühl"-Suche. Hierbei versuchen wir, den Knoten auszuwählen, der uns am nächsten zum Zielknoten erscheint und gleichzeitig die geringsten Kosten hat. Das bedeutet, dass wir immer den Knoten auswählen, der uns am nächsten zum Zielknoten erscheint und gleichzeitig die geringsten Kosten hat. Dies ist jedoch nicht immer optimal, da es sein kann, dass wir einen Knoten auswählen, der uns zwar am nächsten zum Zielknoten erscheint, aber nicht den kürzesten Weg zu diesem Knoten hat.

In [ ]:
pfad = Pfad()
pfad.menschenwalk(knotenliste)
print(pfad)

### Dijkstra
Der Dijkstra-Algorithmus ist ein Algorithmus, der den kürzesten Weg zwischen zwei Knoten in einem Graphen findet. Der Algorithmus funktioniert, indem er die Knoten in einer Prioritätswarteschlange speichert und immer den Knoten auswählt, der die geringsten Kosten hat. Der Algorithmus wird so lange wiederholt, bis wir den Zielknoten erreicht haben oder alle Knoten besucht wurden. 

Den Algorithmus werden wir gleich zunächst an der Tafel durchspielen.

In [ ]:
pfad = Pfad()
pfad.menschenwalk(knotenliste)
print(pfad)

Zum Vergleich ist hier der Algorithmus in Python implementiert:

In [ ]:
dijkstra_graph(adj_mat)